# Sentinel-2 Super-Resolution (4×)

**Upscale Sentinel-2 imagery from 10m/pixel → 2.5m/pixel using SwinIR**

- ✅ Streaming data pipeline (no full scene downloads)
- ✅ Light fine-tuning (≤3 epochs, L1 loss only)
- ✅ Hallucination guardrails

Enable GPU: `Runtime → Change runtime type → T4 GPU`

In [ ]:
# 1. Clone repository & install
!git clone https://github.com/Rishikarnatakam/Klymo.git
%cd Klymo
!pip install -q -r requirements.txt

In [ ]:
# 2. Verify GPU
import torch
assert torch.cuda.is_available(), "GPU not available! Enable it in Runtime settings."
print(f"✓ GPU: {torch.cuda.get_device_name(0)}")
DEVICE = 'cuda'

In [ ]:
# 3. Kaggle authentication
# Go to kaggle.com/settings → Create New API Token → copy the token shown
import os
import getpass

kaggle_username = input("Enter your Kaggle username: ")
kaggle_token = getpass.getpass("Enter your Kaggle API token (hidden): ")

# Create kaggle.json
os.makedirs(os.path.expanduser('~/.kaggle'), exist_ok=True)
with open(os.path.expanduser('~/.kaggle/kaggle.json'), 'w') as f:
    f.write(f'{{"username":"{kaggle_username}","key":"{kaggle_token}"}}')
os.chmod(os.path.expanduser('~/.kaggle/kaggle.json'), 0o600)

print("✓ Kaggle authenticated")

In [ ]:
# 4. Download WorldStrat dataset (streaming compatible patches)
# Clean up previous downloads to prevent unzip prompts
!rm -rf datasets/worldstrat
!kaggle datasets download -d jucor1/worldstrat -p datasets/worldstrat --unzip -o

# Verify download success
import os
if not os.path.exists("datasets/worldstrat/spot") and not os.path.exists("datasets/worldstrat/sentinel2"):
    print("\n❌ DOWNLOAD FAILED / ACCESS DENIED")
    print("You likely encountered a 403 Forbidden error.")
    print("FIX: Please visit https://www.kaggle.com/datasets/jucor1/worldstrat")
    print("Click 'Download' (or 'Agree') to accept the dataset rules, then re-run this cell.")
else:
    print("✓ Dataset ready")

In [ ]:
# 5. Create streaming DataLoader
# Loads patches one-by-one, processes, releases from memory
from src.data.worldstrat_loader import WorldStratDataset
from torch.utils.data import DataLoader
from pathlib import Path

train_ds = WorldStratDataset(
    root_dir=Path('datasets/worldstrat'),
    split='train',
    max_samples=200  # Use subset for efficiency
)

val_ds = WorldStratDataset(
    root_dir=Path('datasets/worldstrat'),
    split='validation',
    max_samples=40
)

# Streaming loader: batch_size=1 → load, train, release
train_loader = DataLoader(train_ds, batch_size=4, shuffle=True, pin_memory=True)
val_loader = DataLoader(val_ds, batch_size=4, shuffle=False, pin_memory=True)

print(f"✓ Train: {len(train_ds)} patches | Val: {len(val_ds)} patches")
print("Data is streamed patch-by-patch, not stored in memory")

In [ ]:
# 6. Fine-tune SwinIR (3 epochs, L1 loss only)
from src.training.finetune import SwinIRTrainer
from src.models.swinir import load_swinir_model

# Load pretrained model
model = load_swinir_model(device=DEVICE)
print(f"✓ Loaded pretrained SwinIR (4×)")

# Create trainer with conservative settings
trainer = SwinIRTrainer(
    model=model,
    device=DEVICE,
    learning_rate=1e-5,  # Low LR for fine-tuning
    max_epochs=3,        # Hard limit per spec
)

# Train (streaming: load patch → train → release memory)
results = trainer.train(train_loader, val_loader)
print(f"\n✓ Fine-tuning complete! Best loss: {results['best_loss']:.4f}")

In [ ]:
# 7. GEE authentication (for live tile fetching)
from google.colab import auth
auth.authenticate_user()

import ee
ee.Initialize(project='klymo-486313')
print("✓ GEE authenticated")

In [ ]:
# 8. Fetch real Sentinel-2 tile (streamed via GEE API)
from src.data.gee_fetcher import GEEFetcher

fetcher = GEEFetcher(authenticate=False)
fetcher.authenticated = True
fetcher.ee = ee

# Stream tile from Delhi, India
delhi_tile = fetcher.fetch_tile('delhi', tile_size=256)
print(f"✓ Streamed tile: {delhi_tile.shape} from GEE (no local storage)")

import matplotlib.pyplot as plt
plt.figure(figsize=(6, 6))
plt.imshow(delhi_tile)
plt.title('Sentinel-2 Delhi (10m/pixel) - Streamed from GEE')
plt.axis('off')
plt.show()

In [ ]:
# 9. Run super-resolution on streamed tile
from src.inference.pipeline import SuperResolutionPipeline

pipeline = SuperResolutionPipeline(device=DEVICE)
results = pipeline.run(delhi_tile)

print(f"Input:  {results['lr'].shape} → 10m/pixel")
print(f"Output: {results['sr'].shape} → 2.5m/pixel (4× enhancement)")

In [ ]:
# 10. Visualize: LR → Bicubic → SwinIR comparison
import matplotlib.pyplot as plt
from src.data.preprocessing import to_8bit_visualization
from skimage.transform import resize

lr = results['lr']
bicubic = results['bicubic']
sr = results['sr']

lr_up = resize(lr, sr.shape[:2], order=0, preserve_range=True)

fig, axes = plt.subplots(1, 3, figsize=(18, 6))

axes[0].imshow(to_8bit_visualization(lr_up))
axes[0].set_title('Low Resolution (10m/pixel)', fontsize=14)
axes[0].axis('off')

axes[1].imshow(to_8bit_visualization(bicubic))
axes[1].set_title('Bicubic 4× (Baseline)', fontsize=14)
axes[1].axis('off')

axes[2].imshow(to_8bit_visualization(sr))
axes[2].set_title('SwinIR 4× (2.5m/pixel)', fontsize=14)
axes[2].axis('off')

plt.tight_layout()
plt.savefig('outputs/visualizations/comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Saved comparison to outputs/visualizations/comparison.png")

In [ ]:
# 11. Compute quality metrics
from src.metrics.psnr import compute_psnr
from src.metrics.ssim import compute_ssim

psnr = compute_psnr(bicubic, sr)
ssim = compute_ssim(bicubic, sr)

print("="*40)
print("Quality Metrics (SwinIR vs Bicubic)")
print("="*40)
print(f"  PSNR: {psnr:.2f} dB")
print(f"  SSIM: {ssim:.4f}")
print("="*40)

# Show hallucination check results
if 'checks' in results:
    print("\nHallucination Checks:")
    for name, check in results['checks']['checks'].items():
        status = "✓" if check['passed'] else "✗"
        print(f"  {status} {name}: {check['score']:.4f}")

In [ ]:
# 12. Download results
from google.colab import files
files.download('outputs/visualizations/comparison.png')
print("✓ Download started!")